In [1]:
import yaml
import os
import sys
import pandas as pd

sys.path.append("../src")  # add parent directory to Python path
from GroupAssignment import GroupAssignmentSimulator
from OutcomeGeneration import OutcomeSimulator
from AttributeGeneration import AttributeSimulator
from ExperimentSimulation import ExperimentSimulator
from SimulationEngine import AnalysisOverhead, ExperimentAnalyzer, BaseSimulator, _worker_run_block
from PostProcessing import ResultsProcessor, ResultsVisualizer
from GeneralDataMaintenance import make_output_folder

In [2]:
yaml_parms = {}
driver_file = "prelim_driver.csv"
if driver_file[(len(driver_file)-5):]==".yaml":
    with open("../configs/main_driver/" + driver_file, "r") as f:
        yaml_parms["driver"] = yaml.safe_load(f)
elif driver_file[(len(driver_file)-4):]==".csv":
    driver_df = pd.read_csv("../configs/main_driver/" + driver_file)
    yaml_parms["driver"] = {}
    for row in driver_df.index:
        driver_row = driver_df.loc[row,:]
        sim_id = driver_row["sim_label"]
        yaml_parms["driver"][sim_id] = {}
        for var in ["run_simulation", "n", "T", "n_iter", "seed"]:
            yaml_parms["driver"][sim_id][var] = int(driver_row[var])
        yaml_parms["driver"][sim_id]["configs"] = {}
        for config_type in ["attribute", "randomization", "analysis"]:
            yaml_parms["driver"][sim_id]["configs"][config_type] = driver_row["configs_"+config_type]
        yaml_parms["driver"][sim_id]["configs"]["outcome"] = {
            "y0":driver_row["configs_outcome_y0"],
            "tau":driver_row["configs_outcome_tau"],
        }

for setting in yaml_parms["driver"].keys():
    if setting=="driver":
        ValueError("Cannot use \'driver\' as simulation id label.")
    yaml_parms[setting] = {}
    for var in ["run_simulation", "n", "T", "n_iter", "seed"]:
        yaml_parms[setting][var] = yaml_parms["driver"][setting][var]
    for config_type in ["attribute", "randomization", "analysis"]:
        with open("../configs/" + config_type + "/" + yaml_parms["driver"][setting]["configs"][config_type], "r") as f:
            yaml_parms[setting][config_type] = yaml.safe_load(f)
    yaml_parms[setting]["outcome"] = {}
    with open("../configs/outcome/" + yaml_parms["driver"][setting]["configs"]["outcome"]["y0"], "r") as f:
        yaml_parms[setting]["outcome"]["y0"] = yaml.safe_load(f)
    with open("../configs/effect/" + yaml_parms["driver"][setting]["configs"]["outcome"]["tau"], "r") as f:
        yaml_parms[setting]["outcome"]["tau"] = yaml.safe_load(f)
    

In [3]:
test_setting = "sim_test"

In [4]:
#yaml_parms
#yaml_parms[test_setting]["analysis"]

In [5]:
outcome_simulator = OutcomeSimulator(yaml_parms=yaml_parms[test_setting]["outcome"])
attribute_simulator = AttributeSimulator(yaml_parms=yaml_parms[test_setting]["attribute"])
group_simulator = GroupAssignmentSimulator(yaml_parms=yaml_parms[test_setting]["randomization"], H=attribute_simulator.H)
analysis_overhead = AnalysisOverhead(yaml_parms=yaml_parms[test_setting]["analysis"], 
                                     n=yaml_parms[test_setting]["n"], T=yaml_parms[test_setting]["T"], 
                                     m=group_simulator.m,
                                     H=group_simulator.H)
simulator = BaseSimulator(analysis_overhead=analysis_overhead, 
                          outcome_simulator=outcome_simulator, 
                          attribute_simulator=attribute_simulator, 
                          group_simulator=group_simulator)

In [6]:
test_results = simulator.run_sims(n_iter=yaml_parms[test_setting]["n_iter"], 
                                  n_thread = 5, 
                                  seed=yaml_parms[test_setting]["seed"])

In [7]:
test_processed_results = ResultsProcessor(test_results)
test_results_visualizer = ResultsVisualizer(test_processed_results)
print(test_results_visualizer.create_basic_summary_table())

             Estimand          Bias     Coverage  RMSE
0      tau(2-0,0-2)^1   0.15 (0.09)  0.94 (0.02)  1.79
1     tau(2-0,0-2)^10   0.04 (0.03)  0.96 (0.01)  0.20
2      tau(2-0,0-2)^2   0.09 (0.07)  0.94 (0.02)  1.10
3      tau(2-0,0-2)^3   0.04 (0.06)  0.94 (0.02)  0.71
4      tau(2-0,0-2)^4   0.04 (0.05)  0.92 (0.02)  0.57
5      tau(2-0,0-2)^5   0.06 (0.05)  0.94 (0.02)  0.47
6      tau(2-0,0-2)^6   0.04 (0.04)  0.94 (0.02)  0.36
7      tau(2-0,0-2)^7   0.05 (0.04)  0.96 (0.01)  0.29
8      tau(2-0,0-2)^8   0.03 (0.04)  0.97 (0.01)  0.25
9      tau(2-0,0-2)^9   0.04 (0.03)  0.95 (0.02)  0.21
10     tau_2(2-0,0-2)   0.04 (0.10)  0.95 (0.02)  1.95
11     tau_3(2-0,0-2)  -0.05 (0.10)  0.94 (0.02)  1.80
12     tau_4(2-0,0-2)   0.01 (0.10)  0.94 (0.02)  2.13
13  tau_[0]1(2-0,0-2)   0.14 (0.14)  0.94 (0.02)  3.68
14  tau_[0]2(2-0,0-2)   0.17 (0.15)  0.93 (0.02)  4.23
15  tau_[0]3(2-0,0-2)  -0.03 (0.14)  0.95 (0.02)  3.87
16  tau_[1]1(2-0,0-2)   0.16 (0.14)  0.94 (0.02)  4.11
17  tau_[1

In [9]:
analysis_overhead_990 = AnalysisOverhead(yaml_parms[test_setting]["analysis"], 
                                     n=990, T=10, 
                                     m=group_simulator.m,
                                     H=group_simulator.H)
simulator_990 = BaseSimulator(analysis_overhead=analysis_overhead_990, 
                          outcome_simulator=outcome_simulator, 
                          attribute_simulator=attribute_simulator, 
                          group_simulator=group_simulator)
test_results_990 = simulator_990.run_sims(n_iter=100, n_thread = 5, seed=657)

In [10]:
test_processed_results_990 = ResultsProcessor(test_results_990)
test_results_visualizer_990 = ResultsVisualizer(test_processed_results_990)
print(test_results_visualizer_990.create_basic_summary_table())

              Estimand          Bias     Coverage  RMSE
0         tau(2-0,0-2)  -0.00 (0.01)  0.95 (0.02)  0.02
1       tau_1(2-0,0-2)   0.01 (0.04)  0.96 (0.02)  0.20
2      tau_10(2-0,0-2)  -0.02 (0.05)  0.93 (0.03)  0.23
3       tau_2(2-0,0-2)  -0.05 (0.04)  0.96 (0.02)  0.16
4       tau_3(2-0,0-2)   0.02 (0.05)  0.94 (0.02)  0.24
5       tau_4(2-0,0-2)   0.01 (0.05)  0.95 (0.02)  0.21
6       tau_5(2-0,0-2)   0.04 (0.05)  0.94 (0.02)  0.21
7       tau_6(2-0,0-2)  -0.02 (0.05)  0.95 (0.02)  0.20
8       tau_7(2-0,0-2)  -0.00 (0.04)  0.99 (0.01)  0.19
9       tau_8(2-0,0-2)   0.02 (0.05)  0.94 (0.02)  0.26
10      tau_9(2-0,0-2)  -0.03 (0.04)  0.97 (0.02)  0.18
11   tau_[0]1(2-0,0-2)  -0.08 (0.07)  0.93 (0.03)  0.47
12  tau_[0]10(2-0,0-2)  -0.07 (0.06)  0.93 (0.03)  0.40
13   tau_[0]2(2-0,0-2)  -0.05 (0.06)  0.96 (0.02)  0.36
14   tau_[0]3(2-0,0-2)  -0.07 (0.07)  0.91 (0.03)  0.54
15   tau_[0]4(2-0,0-2)   0.05 (0.06)  0.95 (0.02)  0.34
16   tau_[0]5(2-0,0-2)   0.01 (0.06)  0.97 (0.02

In [11]:
analysis_overhead_T50 = AnalysisOverhead(yaml_parms[test_setting]["analysis"], 
                                     n=99, T=50, 
                                     m=group_simulator.m,
                                     H=group_simulator.H)
simulator_T50 = BaseSimulator(analysis_overhead=analysis_overhead_T50, 
                          outcome_simulator=outcome_simulator, 
                          attribute_simulator=attribute_simulator, 
                          group_simulator=group_simulator)
test_results_T50 = simulator_T50.run_sims(n_iter=100, n_thread = 5, seed=2713)

In [12]:
test_processed_results_T50 = ResultsProcessor(test_results_T50)
test_results_visualizer_T50 = ResultsVisualizer(test_processed_results_T50)
print(test_results_visualizer_T50.create_basic_summary_table())

               Estimand          Bias     Coverage  RMSE
0          tau(2-0,0-2)  -0.01 (0.02)  0.94 (0.02)  0.04
1        tau_1(2-0,0-2)   0.11 (0.14)  0.94 (0.02)  1.86
2       tau_10(2-0,0-2)   0.07 (0.15)  0.92 (0.03)  2.20
3       tau_11(2-0,0-2)   0.05 (0.14)  0.97 (0.02)  1.81
4       tau_12(2-0,0-2)  -0.15 (0.15)  0.89 (0.03)  2.31
..                  ...           ...          ...   ...
146  tau_[1]50(2-0,0-2)   0.38 (0.21)  0.90 (0.03)  4.35
147   tau_[1]6(2-0,0-2)   0.15 (0.19)  0.92 (0.03)  3.61
148   tau_[1]7(2-0,0-2)  -0.11 (0.20)  0.94 (0.02)  4.00
149   tau_[1]8(2-0,0-2)  -0.10 (0.18)  0.95 (0.02)  3.36
150   tau_[1]9(2-0,0-2)   0.07 (0.18)  0.95 (0.02)  3.25

[151 rows x 4 columns]
